**Data Engineer exercises**
- Date: 9/16/2020
- Purpose: ALS Data Engineer exercise


<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

## Auxiliary code to structure notebook

HTML code to left align markdown tables

In [1]:
%%html
<style>
table {float:left}
</style>

javacript code to update ToC

In [2]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Purpose
This notebook contains exploratory data analysis and solutions to the Anne Lewis Strategies Hiring Data Engineer Exercise.

## Question 1
**Produce a “people” file with the following schema. Save it as a CSV with a header line to the working directory.**

    
| Column | Type | Description |
| :-- | :-- | :-- |
|email | string | Primary email address | 
|code | string | Source code |
|is_unsub | boolean | If primary email address is unsubscribed |
|created_dt | datetime | Person creation datetime |
|updated_dt | datetime | Person updated datetime |



## Question 2
**Use the output of #1 to produce an “acquisition_facts” file with the following schema that aggregates stats about when people in the dataset were acquired. Save it to the working directory.**

| Column | Type | Description | 
| :-- | :-- | :-- |
| acquisition_date | date | Calendar date of acquisition | 
| acquisitions | int | Number of constituents acquired on acquisition date |

# Method Notes
This section contains notes on the methodologies used in data analysis

## Data frame notes

The following dataframes are used to reference the respective Constituent data.

1. `df_info` : [Constituent Information](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv)
2. `df_email` : [Constituent Email Addresses](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv)
3. `df_subs` : [Constituent Subscription Status](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv)


## Data sources
This section details the data sources for the different output files generated.

### People file data source

This section details which dataframe is used to gather the data output in the persons file:

| Column | Type | Description | Dataframe source | 
| :-- | :-- | :-- | :-- |
|email | string | Primary email address | `df_emails.email` |
|code | string | Source code | `df_info.source` |
|is_unsub | boolean | If primary email address is unsubscribed | `df_subs.is_unsub` |
|created_dt | datetime | Person creation datetime | `df_info.create_dt` |
|updated_dt | datetime | Person updated datetime | `df_info.modified_dt` |



### Acquisition facts data source
Data source for the acquisition data file

| Column | Type | Description | Data source |
| :-- | :-- | :-- | :-- |
| acquisition_date | date | Calendar date of acquisition | Date extracted from `created_dt` from people file | 
| acquisitions | int | Number of constituents acquired on acquisition date | Count() extracted from number of emails on `created_dt` |

## Keep relevant columns
To increase notebook performance and readability, I only keep columns that are relevant to the analysis.

Based on the questions outlined in this exercise and the columns identified in [Identify relevant information & entity relationships](#identify-relevant-information-&-entity-relationships), I am interested in the following columns for each dataframe:


**df_info:**
- `cons_id`: primary key, relates to other columns
- `source`: code string data for q1 table
- `create_dt`: created_dt datetime data for q1 table
- `modified_dt`: updated_dt datetime for q1 table

**df_emails:**
- `cons_email_id`: primary key
- `cons_id`: foreign key used to link to df_info
- `email`: email string data for q1 table

**df_subs:**
- `cons_email_chapter_subscription_id`: primary key
- `cons_email_id`: foreign key to link to df_emails
- `isunsub`: is_unsub boolean data for q1 table

## Relational joins
The three data files contain relational data with contain primary and foreign keys to connect information about users. Relational joins are required to retrieve the information for the people file.

I use `df_emails` as the base data file because it contains the most user records.
Left joins to the df_emails table are made to preserve email addresses. This is based on the assumption that email addresses are the variable of interest.

Given df_emails, the following left joins can be made:
- `df_emails.cons_email_id` = `df_subs.cons_email_id`
- `df_emails.cons_id` = `df_info.cons_id`

*NOTE*: Because I used a left_join, it's possible that there will be NULL values in merged columns.

## Handling missing data
The script handles missing data based on the intent of the data. 

Questions 1 and 2 are interested in the following data:
1. `email`
1. `code`
1. `is_unsub`
1. `created_dt`
1. `updated_dt`

`email` is the unique identifier for each row and used to count constituents 'acquired'. Additionally, I use the variable `create_dt` from the constituent information file to record acquisition dates. Because `email` is a primary key and `create_dt` is used to create the acquisition_facts.csv, I will drop records with NULL values in these fields.

The folloiwng data: `code`, `is_unsub`, and `updated_dt` are descriptive information about the constituent. This data is not necessary for the acquisition_facts output for question 2. As such, missing data in these columns will be filled witht he string "unknown".

## Duplicate values
Because `email` is the primary key used in the csv outputs, I remove all duplicates.

## Duplicate foreign keys
In exploratory analysis, I discovered that there are duplicate foriegn keys in `df_emails` and `df_subs`.

### df_emails.cons_id
First, there are duplicate foreign keys in `df_emails.cons_id`. It *is* a possibility that `df_emails.cons_id` to `df_info.cons_id` is a one to many relationship, although unlikely due to there being two datetimes values in each row. 

**Note**: Because analysis shows that 86% of `df_emails` contains duplicate foreign keys, I opt to keep the the duplicates of `df_emails.cons_id` rather than lose valuable data. **This decision will affect the acquisition_facts.csv output**. In the case that the duplicate `df_emails.cons_id` foreign keys _should_ be removed, I left a commented function to remove the duplicate keys.


### df_subs.cons_email_ids
Additionally, there are duplicate foreign keys in `df_subs.cons_email_ids`. For this analysis, I remove the rows with duplicate `cons_email_ids` foreign keys. I remove the duplicate keys to avoid erroneously providing incorrect information on the `inunsub` status of Constituents. Sending emails to unsubscribed Constituents may run into legal issues under the CAN-SPAM Act. 

*Note*: further analysis may explore if rows with duplicate `cons_email_id` contain identical information (i.e., `isunsub`). If so, analysis can keep one instance of each duplicate.

# Libraries

In [3]:
import os
import re
from typing import List, Tuple, Union


import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

## Uncomment below code to install non-standard 3rd party libraries
# missingno used to identify missing data
# pywrangle is an open-source library I am developing for data cleaning
# ! pip install missingno
# ! pip install pywrangle  


import missingno as msno
import pywrangle as pw

header = '#' * 10
print('\n', header, "# Loaded libraries", header, sep = '\n')



##########
# Loaded libraries
##########


# Import Data

The following 3 data sources are used:

1. [Constituent Information](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv)
2. [Constituent Email Addresses](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv)
3. [Constituent Subscription Status](https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv)

*Note*: Boolean columns (including is_primary) in all of these datasets are 1/0 numeric values. 1 means True, 0 means False.

In [4]:
## Urls to download csvs
url_info_csv = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv'
url_email_csv = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv'
url_sub_csv = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv'

url_csvs = (
    url_info_csv,
    url_email_csv,
    url_sub_csv
)

## Csv file names
csv_datafiles = cons_data_csvs = (
    'cons.csv',      ## info
    'cons_email.csv',                      
    'cons_email_chapter_subscription.csv'   
)

In [5]:
def load_df(filename_csv: str, url_csv: str) -> object:
    """Auxiliary function to load csvs into dataframe.
    
    Checks if csv exists in working directory. 
    If not, downloads csv from url.
    """
    file_path = f'./{filename_csv}'
    
    print('\n', '#' * 5, filename_csv)
    if os.path.isfile(file_path):
        print("Reading from working directory.")
        df = pd.read_csv(filename_csv)
    
    else:
        print("Loading data from url. This is a large file, so please be patient.")
        df = pd.read_csv(url_csv)
        
        print(f"\tSaving {filename_csv} to working directory.")
        df.to_csv(path_or_buf = filename_csv)
    
    print(f"""Loaded df for {filename_csv}:
    - {len(df.columns)} columns
    - {len(df)} rows""")
    return df


In [ ]:
## Load all csvs

df_info, df_emails, df_subs = (load_df(csv_datafiles[i], url_csvs[i]) for i in range(len(csv_datafiles)))
all_cons_dfs = (df_info, df_emails, df_subs)

print("\nLoaded all csv data.")


 ##### cons.csv
Loading data from url. This is a large file, so please be patient.
	Saving cons.csv to working directory.
Loaded df for cons.csv:
    - 29 columns
    - 700000 rows

 ##### cons_email.csv
Loading data from url. This is a large file, so please be patient.


# Data cleaning

## Face check data
This section does a face check of the data to look at the contents of each dataframe.


In [ ]:
df_info.head()

In [ ]:
df_emails.head()

In [ ]:
df_subs.head()

## Identify relevant information & entity relationships
This section looks at the schema of each dataframe, identifies relevant columns for the ETL exercise, and primary/foreign keys to connect data frame results.

In [ ]:
df_info.info()

Relevant columns in **df_info**:
- `cons_id`: primary key, relates to other columns
- `source`: code string data for q1 table
- `create_dt`: created_dt datetime data for q1 table
- `modified_dt`: updated_dt datetime for q1 table

In [ ]:
df_emails.info()

Relevant columns in **df_emails**:
- `cons_email_id`: primary key
- `cons_id`: foreign key used to link to df_info
- `email`: email string data for q1 table

*NOTE:*
There are two unclear elements in this table.
- `create_dt`: unclear if this may also fulfill q1 table requirement.
- `modified_dt`: unclear if this may also fulfill q1 table requirement

However, because these are associated in the email information, I will assume that the `create_dt` and `modified_dt` is used for the email, and not the person. As such, I will use the `create_dt` and `modified_dt` in df_info.

In [ ]:
df_subs.info()

Relevant columns in **df_subs**:
- `cons_email_chapter_subscription_id`: primary key
- `cons_email_id`: foreign key to link to df_emails
- `isunsub`: is_unsub boolean data for q1 table

#### Entity relationship notes
The three data files contain relational data with contain primary and foreign keys to connect information about users. Relational joins are required to retrieve the information for the people file.

I use `df_emails` as the base data file because it contains the most user records.
Left joins to the df_emails table are made to preserve email addresses. This is based on the assumption that email addresses are the variable of interest.

Given df_emails, the following left joins can be made:
- `df_emails.cons_email_id` = `df_subs.cons_email_id`
- `df_emails.cons_id` = `df_info.cons_id`

*NOTE*: Because I used a left_join, it's possible that there will be NULL values in merged columns.


## Keep relevant columns
To increase notebook performance and readability, I only keep columns that are relevant to the analysis.

Based on the questions outlined in this exercise and the columns identified in [Identify relevant information & entity relationships](#identify-relevant-information-&-entity-relationships), I am interested in the following columns for each dataframe:


**df_info:**
- `cons_id`: primary key, relates to other columns
- `source`: code string data for q1 table
- `create_dt`: created_dt datetime data for q1 table
- `modified_dt`: updated_dt datetime for q1 table

**df_emails:**
- `cons_email_id`: primary key
- `cons_id`: foreign key used to link to df_info
- `email`: email string data for q1 table

**df_subs:**
- `cons_email_chapter_subscription_id`: primary key
- `cons_email_id`: foreign key to link to df_emails
- `isunsub`: is_unsub boolean data for q1 table

In [ ]:
## columns to save for each df
cols_df_info = [
    'cons_id',
    'source',
    'create_dt',
    'modified_dt'
]
cols_df_email = [
    'cons_email_id',
    'cons_id',
    'email',
#     'create_dt',
#     'modified_dt',
]
cols_df_subs = [
    'cons_email_chapter_subscription_id',
    'cons_email_id',
    'isunsub'
]


In [ ]:
## Auxilary function to modify df and track changes
def keep_df_cols(df: object, columns: list) -> object:
    """Returns pandas dataframe with only specified columns.
    
    Uses pywrangle to document df changes."""
    df_old = pw.record_df_info(df)
    df = df[columns]
    pw.print_df_changes(df, df_old)
    return df


In [ ]:
## Keep information columns
df_info = keep_df_cols(df_info, cols_df_info)
df_info.head()

Drop 25 columns from `df_info` and 86% of the data.

In [ ]:
## Keep email columns
df_emails = keep_df_cols(df_emails, cols_df_email)
df_emails.head()

Drop 13 columns from `df_emails` and 81% of the data.

In [ ]:
## Keep subscription columns
df_subs = keep_df_cols(df_subs, cols_df_subs)
df_subs.head()

Drop 3 columns from `df_subs` and 50% of the data.

## Missing data
Look at the missing data in each dataframe.

In [ ]:
def check_df_missing_data(df: object, df_name: str) -> None:
    """Creates msno.bar graph of data present in dataframe."""
    msno.bar(df)
    plt.title(f"Non-NULL values for {df_name} data.")
    plt.xlabel("Column")
    plt.ylabel("% data present")
    plt.figure(figsize=(5, 2))

### Missing constituent info data

In [ ]:
check_df_missing_data(df_info, "Constituent Information")

NOTE: There `source` column is missing approximately half of its data. Because there is still relevant information on for `create_dt` and `modified_dt`, I will keep missing data in the source columns,

In [ ]:
df_info['source'].value_counts()

Source data is categorical with 4 categories:
- organic
- google
- twitter
- facebook

Because about half of source values are NULL, I will preserve them and label them as `unknown`.

In [ ]:
df_info['source'].fillna("unknown", inplace = True)

Check NULLs are transformed

In [ ]:
check_df_missing_data(df_info, "Constituent Information")

In [ ]:
df_info['source'].value_counts()

### Missing email data

In [ ]:
check_df_missing_data(df_emails, "Constituent Emails")

No missing values in `df_emails`

### Missing subscription data

In [ ]:
check_df_missing_data(df_subs, "Constituent Subscription")

No missing values in `df_subs`.

## Clean string data
This section cleans string data in the data frame columns.

All strings are lower cased in the column for consistency purposes.

### Standardize constituent information
Use pywrangle to standardize string column casing. Pass a tuple of tuples containing a string indicating the column to clean, and an integer representing the cleaning method to use.

In [ ]:
cons_info_strcol_caseint: Tuple[Tuple[ str, int]] = (
    ("source", 0),
    ("create_dt", 0),
    ("modified_dt", 0)
)
df_info = pw.clean_str_columns( df_info, cons_info_strcol_caseint)

### Standardize emails

In [ ]:
cons_email_strcol_caseint: Tuple[Tuple[ str, int]] = (
    ('email', 0),
)
df_emails = pw.clean_str_columns( df_emails, cons_email_strcol_caseint)

### Standardize subscription information

No string data in `df_subs`

## Check duplicates
This section checks for duplicate data entries in the Constituent data

In [ ]:
## Auxiliary function to check duplicates
def check_df_duplicates(df, check_columns: List[str]) -> None:
    """Prints df and df.size of duplicates rows, and duplicates for specified columns."""
    ## duplicate rows
    df_duplicate_rows = df[df.duplicated() == True]
    print(f"Number of df duplicate rows {len(df_duplicate_rows)}")
    
    ## duplicates in column
    for col in check_columns:
        df_col_duplicates = df[df.duplicated(col)]
        print(f"Number of duplicates in {col}: {len(df_col_duplicates)}")
    return None

In [ ]:
def check_percent_missing_data(df, df_dup) -> None:
    """Prints the percentage of missing data."""
    print( (df_dup.size) / df.size * 100)

In [ ]:
## Auxiliary function to remove duplicates
def remove_df_duplicates(df, column: str, keep: Union[str, bool] = False) -> "dataframe":
    """Auxiliary method to remove rows with duplicates n specified column.
    
    Uses pywrangle to record difference in df."""
    old_df = pw.record_df_info(df)
    df.drop_duplicates(subset = column, keep = keep, inplace = True)
    pw.print_df_changes(df, old_df)
    return df
    

### info duplicates

In [ ]:
print(df_info.columns)

In [ ]:
check_info_cols = ['cons_id']   # no duplicate primary keys.
check_df_duplicates(df_info, check_columns = check_info_cols)

### email duplicates

In [ ]:
print(df_emails.columns)

In [ ]:
check_email_cols = ['cons_email_id', 'cons_id']   # No duplicate primary or foreign keys
check_df_duplicates(df_emails, check_columns = check_email_cols)

In [ ]:
dup_col = 'cons_id'
df_dup_emails = df_emails[df_emails.duplicated(dup_col, keep = False)].sort_values(dup_col)
df_dup_emails.head()

In [ ]:
df_info.sort_values(dup_col).head()

There are a number of duplicate foreign keys for cons_id and we cannot be sure which user should reference the cons_id.

It *is* a possibility that `df_emails.cons_id` to `df_info.cons_id` is a one to many relationship, although unlikely due to there being two datetimes values in each row. 

In [ ]:
check_percent_missing_data(df_emails, df_dup_emails)

*Note*: Because analysis shows that 86% of the data contains duplicate foreign keys, I opt to keep the the duplicates of `df_emails.cons_id` rather than losing valuable data.

In [ ]:
# df_emails = remove_df_duplicates(df_emails, dup_col, keep = False)

### subscription duplicates

In [ ]:
print(df_subs.columns)

In [ ]:
check_sub_cols = ['cons_email_chapter_subscription_id', 'cons_email_id']  # primary and foreign keys
check_df_duplicates(df_subs, check_columns= check_sub_cols)

In [ ]:
dup_col = 'cons_email_id'
df_dup_subs = df_subs[ df_subs.duplicated(dup_col, keep = False)].sort_values(dup_col)
df_dup_subs.head()

There are duplicate foreign keys in `df_subs.cons_email_ids`. For this analysis, I remove the rows with duplicate `cons_email_ids` foreign keys. I remove the duplicate keys to avoid erroneously providing incorrect information on the `inunsub` status of Constituents. Sending emails to unsubscribed Constituents may run into legal issues under the CAN-SPAM Act.

Note: further analysis may explore if rows with duplicate `cons_email_id` contain identical information (i.e., `isunsub`). If so, analysis can keep one instance of each duplicate.

In [ ]:
check_percent_missing_data(df_subs, df_dup_subs)

In [ ]:
df_subs = remove_df_duplicates(df_subs, dup_col, keep = False)

In [ ]:
## Sanity check that the rows from `df_subs` == `len(df_dup_subs)`
len(df_dup_subs)

## Clean datettime
datetime data stored in df_info is stored in the {Day, Year-Month-Day Hour:Minute:Second} format. 
I convert this into ISO-datetime format for convenience.

In [ ]:
df_info.head()

In [ ]:
## Auxiliary function to get datetime from day, datetime information stored in
def get_datetime(day_datetime: str) -> 'datetime':
    """Returns datetime object from str datetime.
    
    >>> get_datetime('sat, 2017-09-30 08:26:54') 
    2017-09-30 08:26:54
    """
    int_index = re.search(r'\d', day_datetime).start()
    datetime_list = list(day_datetime)[int_index:]
    return ''.join(datetime_list)

In [ ]:
## Specify date-time format parser
datetime_format = "%Y-%m-%d %H:%M:%S"

## Cols to iterate through
datetime_cols = ('create_dt', 'modified_dt')

for col in datetime_cols:
    df_info[col] = df_info[col].map(get_datetime)
    df_info[col] = pd.to_datetime(df_info[col], format = datetime_format)
    

In [ ]:
df_info.head()

In [ ]:
df_info.info()

`create_dt` and `modified_dt` are now registered as datetime64 objects.

# Join data
In this section, I join the three dataframes into a single dataframe with the desired information for quesiton 1:
- email
- code
- is_unsub
- created_dt
- updated_dt

Given df_emails, the following left joins are made:
- df_emails.cons_email_id = df_subs.cons_email_id
- df_emails.cons_id = df_info.cons_id

In [ ]:
## Left Join df_emails and df_info
df_email_info = pd.merge(left = df_emails, right = df_info, how= 'left', left_on = 'cons_id', right_on = 'cons_id')
df_email_info.head()

In [ ]:
## Create total data_frame for q1
df_email_info_subs = pd.merge(
    left = df_email_info, right = df_subs, 
    how = 'left', 
    left_on = 'cons_email_id', right_on = 'cons_email_id')
df_email_info_subs.sort_values(by = 'cons_id', inplace = True)

In [ ]:
df_email_info_subs.head()

# Q1: 'People' File
This section saves the 'people' file as a csv with a header line.

In [ ]:
desired_columns = [
    'email',
    'source',
    'isunsub',
    'create_dt',
    'modified_dt'
]
df_q1 = df_email_info_subs[desired_columns]
df_q1.reset_index(drop = True, inplace = True)
df_q1.head()

In [ ]:
## Rename columns
q1_col_names = (
    'email',
    'code',
    'is_unsub',
    'created_dt',
    'updated_dt'
)
df_q1.columns = q1_col_names
df_q1.head()

In [ ]:
check_df_missing_data(df_q1, "Dataframe for q1")

Note: The left join on emails to subcription status created NULL values in the `in_unsub` column.

Because the second question does not specify to remove people who are unsubbed, I will preserve rows with NULL values in the `in_unsub` column. I will mark these values as `unknown`.

In [ ]:
missing_col = 'is_unsub'
df_q1[missing_col].fillna("unknown", inplace = True)

In [ ]:
check_df_missing_data(df_q1, "Dataframe for q1")

In [ ]:
file_name = 'people.csv'

df_q1.to_csv(
    path_or_buf = file_name,
    index = False,)

print(f"Wrote dataframe to {file_name}")

# Q2: “Acquisition_facts” file with stats about when people where acquired

## Extract date from created_dt
`created_dt` is stored as datetime. This analysis only wants the date.

In [ ]:
df_q1.head()

In [ ]:
df_q1['acquired'] = df_q1['created_dt'].dt.date

In [ ]:
df_q1.head()

## Group values by date
This section creates a dataframe with count values for each acquired date.

In [ ]:
group_var = ['acquired']

df_q2 = (
    df_q1
    .groupby( group_var, as_index = True)
    .count()
    .sort_values(group_var, ascending = True)['email']
    .reset_index()
)


df_q2.head()

Note: date starts at Epoch time - likely reflects the fake data.

In [ ]:
print(len(df_q2))

## Write to csv

In [ ]:
file_name = "acquisition_facts.csv"

df_q2.to_csv(
    path_or_buf = file_name,
    index = False
)